In [1]:
!pip install pandas-datareader

### 1. Importing libraries

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


### 2. Building the AI Trader model

In [0]:
class AI_Trader():
  
    def __init__(self, state_size, action_space=3, model_name="AITrader"): # 3 Actions -> Stay, Buy, Sell

        self.state_size = state_size
        self.action_space = action_space 
        self.memory = deque(maxlen=2000) 
        self.inventory = []              # Stores stocks we have bought, since we can't sell a stock we don't own
        self.model_name = model_name

        # Discount. Helps in maximizing reward over time.
        self.gamma = 0.95
        # Epsilon is used to determine if actions are performed by agent randomly or not. Epsilon=1 means randomly, so model
        # doesn't learn. Some actions should be random for environment exploration, so we don't set epsilon_final=0.
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995

        self.model = self.model_builder()
    
    def model_builder(self):
        # We set activation="linear" as we modify action based on reward, which is a continuous number.
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))
        model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))

        return model

    def trade(self, state):
        """
        Takes in state as input and generates random number. If random number <= epsilon, generate random action, 
        else return action predicted by model using state as input.
        """
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)

        actions = self.model.predict(state)
        return np.argmax(actions[0])
  
  
    def batch_train(self, batch_size):
        batch = []
        # We do this for loop as in time series data, we do't want to randomly access data.
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])

        for state, action, reward, next_state, done in batch:
            # We check if model is in terminal state. If it isn't, then we use bellman eqn. to calculate reward.
            reward = reward
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            
            # We replace the action by the reward. This is why we used MSE as the loss.
            target = self.model.predict(state)
            target[0][action] = reward

            self.model.fit(state, target, epochs=1, verbose=0)
            
        # Decrease epsilon by a factor when it crosses a threshold value.
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

### 3. Data loading and preprocessing

In [0]:
def dataset_loader(stock_name):
    #Complete the dataset loader function
    dataset = data_reader.DataReader(stock_name, data_source="yahoo")
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    close = dataset["Close"]

    return close

In [0]:
def sigmoid(x):
    """
    We use sigmoid function, as we want data in time series to be scaled properly. For example, difference in price
    jumps will be scaled.
    """
    return 1 / (1 + math.exp(-x))

In [0]:
def stocks_price_format(n):
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

In [0]:
def state_creator(data, timestep, window_size):
    """
    data -> actual data
    timestep -> day for which we want to predict the stock price
    window_size -> how many previous observations to consider before making a prediction
    """
    starting_id = timestep - window_size + 1
    if starting_id >= 0:
        windowed_data = data[starting_id:timestep+1]
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
    state = []
    for i in range(window_size - 1):
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))

    return np.array([state])

In [0]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

### 4. Training

In [0]:

window_size = 10
# In Reinforcement Learning, word used for epochs is episodes
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

trader = AI_Trader(window_size)

In [0]:
for episode in range(1, episodes + 1):
    
    print("Episode: {}/{}".format(episode, episodes))
    state = state_creator(data, 0, window_size + 1)
    total_profit = 0
    trader.inventory = []
  
    for t in tqdm(range(data_samples)):
        action = trader.trade(state)
        next_state = state_creator(data, t+1, window_size + 1)
        reward = 0

        if action == 1: #Buying
            trader.inventory.append(data[t])
            print("AI Trader bought: ", stocks_price_format(data[t]))
      
        elif action == 2 and len(trader.inventory) > 0: #Selling
            buy_price = trader.inventory.pop(0)
      
            reward = max(data[t] - buy_price, 0)    # Zero reward if loss
            total_profit += data[t] - buy_price
            print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - 
                                                                                                      buy_price) )
      
        if t == data_samples - 1:
            done = True
        else:
            done = False
      
        trader.memory.append((state, action, reward, next_state, done))
        state = next_state
    
        if done:
            print("########################")
            print("TOTAL PROFIT: {}".format(total_profit))
            print("########################")
    
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
      
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/1258 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 125.150002
AI Trader sold:  $ 128.949997  Profit: $ 3.799995
AI Trader bought:  $ 128.699997
AI Trader bought:  $ 125.800003
AI Trader sold:  $ 125.010002  Profit: - $ 3.689995
AI Trader bought:  $ 125.260002
AI Trader sold:  $ 127.620003  Profit: $ 1.820000
AI Trader bought:  $ 126.320000
AI Trader bought:  $ 128.770004
AI Trader bought:  $ 130.190002
AI Trader bought:  $ 130.059998
AI Trader sold:  $ 131.389999  Profit: $ 6.129997
AI Trader sold:  $ 132.539993  Profit: $ 6.219994
AI Trader sold:  $ 130.279999  Profit: $ 1.509995
AI Trader sold:  $ 130.539993  Profit: $ 0.349991
AI Trader bought:  $ 129.960007
AI Trader sold:  $ 130.119995  Profit: $ 0.059998
AI Trader bought:  $ 129.360001
AI Trader sold:  $ 128.649994  Profit: - $ 1.310013
AI Trader sold:  $ 127.419998  Profit: - $ 1.940002


  3%|▎         | 34/1258 [00:04<13:21,  1.53it/s]

AI Trader bought:  $ 126.919998


  3%|▎         | 38/1258 [00:12<35:08,  1.73s/it]

...

 72%|███████▏  | 905/1258 [29:53<12:40,  2.15s/it]

AI Trader bought:  $ 174.240005


 72%|███████▏  | 907/1258 [29:58<12:34,  2.15s/it]

AI Trader sold:  $ 179.550003  Profit: $ 4.930008


 72%|███████▏  | 908/1258 [30:00<12:29,  2.14s/it]